# Spark (with PySpark)


In this tutorial, you will learn how to use Apache Spark, a framework for large-scale data processing, within a notebook.
Upon completing this lab you will be able to : 
 + Program in Spark with the Python Language
 + Demonstrate how to read and process data using Spark
 + Compare and contrast RDD and Dataframes. 
 + Build a simple machine learning application with Spark.
 
 
 
# Requirements : Installing Spark, PySpark and configuration to run it on jupyter notebooks

According to your OS, you have to follow the following tutorial :

+ [Windows](https://changhsinlee.com/install-pyspark-windows-jupyter/)
+ [Mac Os X](https://jmedium.com/pyspark-in-mac/)
 
At the end of the installation, you should be able to run the following code that is a kind of Hello word in PySpark.

In [ ]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.sql('''select 'spark' as hello ''')
df.show()

In [ ]:
spark.stop()

## Part 1 :  Spark Context

When writing a spark program, the first thing to do is to define a `SparkContext`. 

In Spark, communication occurs between a driver and executors. The driver has Spark jobs that it needs to run and these jobs are split into tasks that are submitted to the executors for completion. The results from these tasks are delivered back to the driver.

Here, we will use the `findspark` package that has to be installed using the following command: 

` pip3 install findspark`

Then we can use the `findspark.init()`function to locate the Spark process.




In [ ]:
import findspark

findspark.init("/Users/talboth/Downloads/spark-2.3.2-bin-hadoop2.7")

import pyspark

sc=pyspark.SparkContext("local[*]","First program in Spark")

Try printing out sc to see its type.

In [ ]:
# TO DO

You can use Python's `dir()` function to get a list of all the attributes (including methods) accessible through the `sc` object.

In [ ]:
dir(sc)

## Part 2 : Understanding Spark RDD’s

### WordCount in Spark
In this part, we will write the wordcount in Spark and apply it on the novel Dracula of Bram Stocker (from the Gutemberg project).

First read the [pg345.txt](./SparkData/pg345.txt) file.

In [ ]:
data = sc.textFile('./SparkData/pg345.txt')
type(data)

To see the content, of the file, we need to run the action `collect` on the  RDD `data`

In [ ]:
data.collect()

With the RDD `data`, from the previous cell, execute a `flatMap()` for each line in the input and then convert it to lower case, remove the commas, split the words on a space and store in the RDD `words`

In [ ]:
# TO DO  - map step

Execute the command with the action `take()` and retrieve the first 10 words from the `flatMap()`transformation

In [ ]:
# TO DO 

Perform a classic `map()` to create a tuple where each word has a count of 1

In [ ]:
# TO DO


Write the reducing function

In [ ]:
# TO DO 


Get all words that occur more than once and sort them alphabetically:

In [ ]:
# TO DO


Now, get all words that occur more than once and sort them by frequency. Select the first 20. Hints : look at the `takeOrdered` action.

In [ ]:
# TO DO 


### A simple exercice

+ Create a Python collection of 10,000 integers
+ Create a Spark base RDD from that collection
+ Subtract one from each value using map
+ Perform action collect to view results
+ Perform action count to view counts
+ Apply transformation filter and view results with collect

In [ ]:
# TO DO

### An improved WordCount

Print the top 10 most frequent words with their probability of appearance


In [ ]:
# TO DO

Get rid of special characters (.,:!?')


In [ ]:
# TO DO

Identify the transformations and the actions in your script


In [ ]:
# TO DO

How many times are the transformations evaluated? (Hint: it depends)


In [ ]:
# TO DO

Can you reduce this number? (Hint: check out "persist")

In [ ]:
# TO DO

In [ ]:
sc.stop()

## Part 3 : Spark SQL and dataframes

In this part, you will explore Spark DataFrames and the SQL Context. In particular, we will work on a database that contains a sample of the world population by working on data that comes from [pplapi](http://pplapi.com/). The file [agents.json](./SparkData/agents.json) is a file that was extracted from this api using the following command :


`wget https://s3-eu-west-1.amazonaws.com/course.oc-static.com/courses/4297166/agents.json`

In [ ]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read.json("./SparkData/agents.json")

Print the 5 first lines of the dataframe `df`

In [ ]:
# TO DO

Write the spark instructions that enable to display the number of French agents.

In [ ]:
# TO DO

Write the spark instructions that enable to display the number of Indian female agents.

In [ ]:
# TO DO

Using the notion of temporary view (function `createTempView`), create a temporary view associated to the dataframe `df`. 

In [ ]:
df.createTempView("temp_table")


Write some SQL query on the resulting table as shown in the example below.

In [ ]:
spark.sql("SELECT country_name,id FROM agents_table ORDER BY id DESC LIMIT 10").show()

In [ ]:
# TO DO - Write some requests.

In [ ]:
spark.stop()
sc.stop()

You can also create a dataframe from an existing RDD as shown on the example below

In [ ]:
sc=pyspark.SparkContext("local[*]","Dataframe examples in Spark")
sqlContext = pyspark.SQLContext(sc)



In [ ]:
documents_rdd = sc.parallelize([
        [1, 'cats are cute', 0],
        [2, 'dogs are playfull', 0],
        [3, 'lions are big', 1],
        [4, 'cars are fast', 1]])
users_rdd = sc.parallelize([
        [0, 'Alice', 20],
        [1, 'Bob', 23],
        [2, 'Charles', 32]])

In [ ]:
documents_df = documents_rdd.toDF(['doc_id', 'text', 'user_id'])
users_df = users_rdd.toDF(['user_id', 'name', 'age'])

# printing the inferred schema for documents
documents_df.printSchema()


Some functions can be apply to a column or to different columns. Here, we compute the age avarage in the dataframe `users_df`. 

In [ ]:
from pyspark.sql import functions as fn

user_age_df = users_df.select(fn.avg('age')).show()


Compute the max of age in the dataframe `users_df`. 

In [ ]:
# TO DO 

Join (as in SQL) the two dataframes `users_df` and `documents_df`

In [ ]:
# TO DO 


Outer Join (Left) (as in SQL) the two dataframes `users_df` and `documents_df`

In [ ]:
# TO DO

## Part 4 : MLlib - Spark for data analysis and machine learning

In this part, we will see how to create a text classifiation application with Spark. We will use some data from the newsgroups [Usenet](https://en.wikipedia.org/wiki/Usenet_newsgroup) and the objective is to predict the topic of the news.
 
The train data is [here](./SparkData/20ng-train-all-terms.txt) and the test data [here](./SparkData/20ng-test-all-terms.txt).
The are obtained using the following commands: 
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-train-all-terms.txt`
and
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-test-all-terms.txt`

Load these data as some RDDs and translate them in DataFrames.

In [ ]:
sc.stop()

findspark.init()
from pyspark.ml.feature import CountVectorizer

sc=pyspark.SparkContext("local[*]","machine learning in Spark")
sqlContext = pyspark.SQLContext(sc)

from pyspark.sql import Row

def load_dataframe(path):
# TO COMPLETE

train_data = load_dataframe("./SparkData/train-all-terms.txt")
test_data = load_dataframe("./SparkData/test-all-terms.txt")

train_data.show()
test_data.show()

A first step is to represent our data, i.e. the messages in the form of a bag-of-word representation using the spark method `CountVectorizer` documented [here](https://spark.apache.org/docs/2.1.0/ml-features.html#countvectorizer)

In [ ]:
# TO DO  - bag of word representation of test and train data

Displayong of the distint labels in the datasets

In [ ]:
train_data.select("label").distinct().sort("label").show(truncate=False)

We will now apply a [NaiveBayes](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes) classifier to our problem. 
Take the time to read the doc and apply it to our problem. You will first have to associate a number to each label. You can use the [`String Indexer`](https://spark.apache.org/docs/2.1.0/ml-features.html#stringindexer) function of pyspark for that.


In [ ]:
# TO DO - Transformation of the label into a number



In [ ]:
# TO DO - Application of the NaivesBayes models


In [ ]:
# TO DO - Application of the trained model to the test dataset and displaying the prediction compared to the real value for the 20 first messages



The Evaluation of the learned model can be done by using the [evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html) module of MLlib. Print the accuracy of the obtained model.

In [ ]:


# TO DO
